### **Carga de datos y procesamiento de texto para BERT**



BERT (Bidirectional Encoder Representations from Transformers) ha revolucionado las tareas de procesamiento de lenguaje natural al capturar información contextual tanto del contexto izquierdo como del derecho. 

Para aprovechar el poder de BERT, incluiremos diversos temas, incluidos la selección aleatoria de muestras, la tokenización, la construcción de vocabulario, el enmascaramiento de texto y la preparación de datos para las tareas de modelo de lenguaje enmascarado (MLM) y predicción de la siguiente oración (NSP). 

#### **Configuración**

In [ ]:
#! pip install 'portalocker>=2.0.0'
#! pip install 'torchtext==0.16.0'
#! pip install transformers==4.39.1
#! pip install pandas

#### **Importando librerías requeridas**


En esta sección importarás las librerías y módulos necesarios para preparar tu conjunto de datos para el entrenamiento con PyTorch. El enfoque está en el procesamiento de texto y en la creación de data loaders que se emplearán para entrenar tus modelos.

* **DataLoader**: una utilidad de PyTorch que te permite cargar datos en lotes, facilitando la gestión de grandes volúmenes de información durante el entrenamiento.
* **build\_vocab\_from\_iterator**: función de `torchtext.vocab` que crea un objeto de vocabulario a partir de un iterador. Este vocabulario es clave para el procesamiento de texto, ya que mapea tokens (palabras) a números enteros.
* **Vocab**: representa el vocabulario, es decir, la asignación de tokens a índices. Se utiliza para convertir datos de texto en una forma numérica que el modelo puede entender.
* **Tensor, torch, nn, Transformer**: módulos y clases centrales de PyTorch para trabajar con tensores (la estructura de datos fundamental en PyTorch), capas de redes neuronales y la arquitectura de modelo Transformer.
* **get\_tokenizer**: función de `torchtext.data.utils` que devuelve un tokenizador para convertir cadenas de texto en listas de tokens.
* **pad\_sequence**: utilidad de `torch.nn.utils.rnn` que rellena secuencias para que todas tengan la misma longitud, un requisito común al procesar lotes de datos con secuencias de longitud variable.

Esta configuración es esencial para procesar datos de texto, convertirlos a formato numérico y preparar lotes de datos para entrenar redes neuronales, especialmente en tareas como modelado de secuencias y clasificación.


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import Vocab
from torch import Tensor
from torch.nn import Transformer
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from itertools import chain
from itertools import islice
from torchtext.datasets import IMDB
from copy import deepcopy
import random
import csv
import json
from tqdm import tqdm
import pandas as pd

# También puedes usar esta sección para suprimir las advertencias generadas por tu código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

#### **Tokenización y construcción del vocabulario**


#### **Tokenización**

* El `tokenizer` se inicializa para tokenizar texto usando reglas básicas de tokenización en inglés, convirtiendo muestras de texto en listas de tokens.

* `yield_tokens` es una función generadora que itera a través de los datos, devolviendo versiones tokenizadas de las muestras de texto. Esta función facilita la construcción del vocabulario al proporcionar un flujo continuo de tokens.

* `word_dict` define los tokens especiales utilizados en el procesamiento de texto, como padding `[PAD]`, inicio de clase `[CLS]`, separador `[SEP]`, máscara `[MASK]` y desconocido `[UNK]`, con sus índices correspondientes.

* Los símbolos especiales y sus índices están definidos explícitamente para mayor claridad y se utilizan en toda la preparación de los datos.

* Las funciones `text_to_index` y `index_to_en` son convertidores auxiliares. La primera convierte texto en una lista de índices numéricos basados en el vocabulario, y la segunda invierte este proceso, traduciendo una secuencia de índices de vuelta a texto legible en inglés.

* **`CLS` (token de clasificación)**: Este token sirve como marcador de *inicio de secuencia (SOS)*. Representa el significado global de toda la oración. Se emplea comúnmente en tareas que requieren comprender la entrada completa, como la clasificación.

* **`SEP` (token separador)**: Utilizado como marcador de *fin de secuencia (EOS)**. También actúa como delimitador en escenarios donde el modelo necesita entender y diferenciar varias oraciones, por ejemplo en tareas de pregunta-respuesta o pares de oraciones.

* **`PAD` (Token de relleno)**: Este token se añade a las secuencias para garantizar que todas las entradas tengan la misma longitud. Durante el entrenamiento, es importante notar que el token `[PAD]`, típicamente con ID 0, no contribuye a los cálculos de gradiente.

* **`MASK` (Token enmascarado)**: Utilizado para reemplazo de palabras en tareas como el modelado de lenguaje enmascarado. Permite a los modelos predecir la identidad de las palabras enmascaradas, facilitando el aprendizaje de representaciones bidireccionales.

* **`UNK` (Token desconocido)**: Actúa como marcador para palabras que no se encuentran en el vocabulario del tokenizador. Este token reemplaza cualquier elemento desconocido o fuera de vocabulario en los datos de entrada.

Estos componentes son fundamentales para el preprocesamiento de datos de texto, asegurando que estén en el formato correcto para el entrenamiento de modelos, incluyendo la tokenización, la conversión numérica y el manejo de tokens especiales necesarios para modelos como BERT.


In [ ]:
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for label, data_sample in data_iter:
        yield tokenizer(data_sample)

# Define símbolos especiales e índices
PAD_IDX, CLS_IDX, SEP_IDX, MASK_IDX, UNK_IDX = 0, 1, 2, 3, 4

# Asegurate de que los tokens estén en el orden de sus índices para insertarlos correctamente en el vocabulario
special_symbols = ['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']


#### **Construcción de vocabulario**

Esta sección se centra en construir el vocabulario a partir del conjunto de datos IMDB.

* Puedes utilizar el conjunto de datos `IMDB` de `torchtext.datasets`, dividiéndolo en conjuntos de entrenamiento y de prueba.
* El vocabulario se crea con la función `build_vocab_from_iterator`, incorporando símbolos especiales (`[PAD]`, `[CLS]`, `[SEP]`, `[MASK]`, `[UNK]`) al principio.
* Se asigna `UNK_IDX` como índice predeterminado para las palabras desconocidas y se muestra por pantalla el tamaño total del vocabulario.

In [ ]:
# crea divisiones de datos
train_iter, test_iter = IMDB(split=('train', 'test'))
all_data_iter = chain(train_iter, test_iter)
# comprueba el tokenizador
# lista(yield_tokens(all_data_iter))[5][:20]
fifth_item_tokens = next(islice(yield_tokens(all_data_iter), 5, None))
print(fifth_item_tokens[:20])


In [ ]:
# crea vocabulario: el vocabulario se construye únicamente usando datos de entrenamiento
vocab = build_vocab_from_iterator(yield_tokens(all_data_iter), specials=special_symbols, special_first=True)

vocab.set_default_index(UNK_IDX)
VOCAB_SIZE = len(vocab)
print(VOCAB_SIZE)

Ahora, crea funciones que transformen los índices de los tokens en sus textos y viceversa. Las usarás más adelante.

In [ ]:
text_to_index=lambda text: [vocab(token) for token in tokenizer(text)]
index_to_en = lambda seq_en: " ".join([vocab.get_itos()[index] for index in seq_en])

Vamos a comprobar los mapeos:


In [ ]:
seq_en = [0, 1, 2, 3, 4, 5, 6]  # Secuencia de entrada de ejemplo
english_sentence = index_to_en(seq_en)
seq2 = [6, 16, 26131]
english_sentence = index_to_en(seq2)

print(english_sentence)

text = "I've seen R-rated films with male nudity. Nowhere, because they don't exist."  # Texto de entrada de ejemplo
text_to_index = lambda text: [vocab[token] for token in tokenizer(text)]
index_sequence = text_to_index(text)

print(index_sequence)

#### **Enmascaramiento de texto y preparación de datos para BERT**

Esta sección presenta funciones para preparar los datos para las tareas de **modelado de lenguaje enmascarado (MLM)** y **predicción de la siguiente oración (NSP)** de BERT, pasos cruciales para ajustar finamente BERT en tareas específicas de NLP.



#### **Enmascaramiento de texto**

La función `Masking` aplica la estrategia MLM de BERT, decidiendo si cada token en una secuencia debe ser enmascarado, dejado sin cambios o reemplazado por un token aleatorio. Este proceso es esencial para entrenar al modelo a predecir palabras enmascaradas basándose en su contexto.



Primero, define una función que devuelva aleatoriamente `0` o `1` siguiendo una distribución de Bernoulli para el muestreo.


In [ ]:
def bernoulli_true_false(p):
    # Crea una distribución de Bernoulli con probabilidad p
    bernoulli_dist = torch.distributions.Bernoulli(torch.tensor([p]))
    # Muestra de esta distribución y convierte 1 en True y 0 en False
    return bernoulli_dist.sample().item() == 1

Ahora, define la función de enmascaramiento:


In [ ]:
def Masking(token):
    # Decide si enmascarar este token (20% de probabilidad)
    mask = bernoulli_true_false(0.2)

    # Si mask es False, retornar inmediatamente con etiqueta '[PAD]'
    if not mask:
        return token, '[PAD]'

    # Si mask es True, continuar con las siguientes operaciones
    # Decide aleatoriamente la operación (50% de probabilidad cada una)
    random_opp = bernoulli_true_false(0.5)
    random_swich = bernoulli_true_false(0.5)

    # Caso 1: si mask, random_opp y random_swich son True
    if mask and random_opp and random_swich:
        # Reemplaza el token con '[MASK]' y asignar una etiqueta de token aleatorio
        mask_label = index_to_en(torch.randint(0, VOCAB_SIZE, (1,)))
        token_ = '[MASK]'

    # Caso 2: si mask y random_opp son True, pero random_swich es False
    elif mask and random_opp and not random_swich:
        # Deja el token sin cambios y asignar como etiqueta el mismo token
        token_ = token
        mask_label = token

    # Caso 3: si mask es True, pero random_opp es False
    else:
        # Reemplaza el token con '[MASK]' y asignar como etiqueta el token original
        token_ = '[MASK]'
        mask_label = token

    return token_, mask_label


Veamos cómo funciona la estrategia de enmascaramiento aleatorio:


In [ ]:
# Fija semilla para garantizar reproducibilidad
torch.manual_seed(100)

# Ejecuta 10 veces el proceso de enmascarado
for l in range(10):
    token = "apple"  # Token original
    token_, label = Masking(token)  # Aplicar función de enmascarado

    # Si el token permanece igual y la etiqueta es [PAD]
    if token == token_ and label == "[PAD]":
        print(
            token_,
            label,
            f"\t El token actual *{token}* permanece sin cambios"
        )

    # Si el token se enmascara y la etiqueta coincide con el token original
    elif token_ == "[MASK]" and label == token:
        print(
            token_,
            label,
            f"\t El token actual *{token}* se enmascara con '{token_}'"
        )

    # En cualquier otro caso, se ha reemplazado por un token aleatorio
    else:
        print(
            token_,
            label,
            f"\t El token actual *{token}* se reemplaza por el token aleatorio #{label}#"
        )

#### **Preparación de datos para MLM**

`prepare_for_mlm` prepara el texto tokenizado para el entrenamiento de MLM aplicando la estrategia de enmascaramiento. Devuelve secuencias de tokens enmascarados junto con sus etiquetas correspondientes, e incluye opcionalmente los tokens originales (sin procesar) como referencia.


In [ ]:
def prepare_for_mlm(tokens, include_raw_tokens=False):
    """
    Prepara el texto tokenizado para el entrenamiento del modelo de lenguaje enmascarado (MLM) de BERT.
    """
    bert_input = []      # Lista para almacenar oraciones procesadas para el MLM de BERT
    bert_label = []      # Lista para almacenar las etiquetas de cada token (enmascarado, aleatorio o sin cambio)
    raw_tokens_list = [] # Lista para almacenar los tokens originales si se requiere
    current_bert_input = []
    current_bert_label = []
    current_raw_tokens = []

    for token in tokens:
        # Aplica la estrategia de enmascaramiento de BERT al token
        masked_token, mask_label = Masking(token)

        # Añade el token procesado y su etiqueta a las listas actuales
        current_bert_input.append(masked_token)
        current_bert_label.append(mask_label)

        # Si se incluyen los tokens originales, añádelos a la lista correspondiente
        if include_raw_tokens:
            current_raw_tokens.append(token)

        # Verifica si el token es un delimitador de oración (., ?, !)
        if token in ['.', '?', '!']:
            # Si la oración actual tiene más de dos tokens, considérala válida
            if len(current_bert_input) > 2:
                bert_input.append(current_bert_input)
                bert_label.append(current_bert_label)
                # Si se incluyen los tokens originales, añádelos a la lista principal
                if include_raw_tokens:
                    raw_tokens_list.append(current_raw_tokens)

                # Reinicia las listas para la siguiente oración
                current_bert_input = []
                current_bert_label = []
                current_raw_tokens = []
            else:
                # Si la oración es muy corta, descártala y reinicia las listas
                current_bert_input = []
                current_bert_label = []
                current_raw_tokens = []

    # Agrega cualquier token restante como una oración si hay alguno
    if current_bert_input:
        bert_input.append(current_bert_input)
        bert_label.append(current_bert_label)
        if include_raw_tokens:
            raw_tokens_list.append(current_raw_tokens)

    # Devuelve las listas preparadas para el entrenamiento MLM de BERT
    return (bert_input, bert_label, raw_tokens_list) if include_raw_tokens else (bert_input, bert_label)


Ahora, veamos cómo las preparaciones de MLM transforman la entrada cruda en la entrada lista para el entrenamiento:


In [ ]:
import torch

# Fija la semilla de PyTorch en 100 para reproducibilidad
torch.manual_seed(100)

# Entrada original
original_input = "The sun sets behind the distant mountains."

# Tokeniza la entrada
tokens = tokenizer(original_input)

# Prepara los tensores para el enmascarado de lenguaje (sin incluir la lista de tokens sin procesar)
bert_input, bert_label = prepare_for_mlm(tokens, include_raw_tokens=False)

# Muestra los resultados sin los tokens crudos
print("Sin tokens crudos:\t",
      "\n\tEntrada original:\t", original_input,
      "\n\tbert_input:\t\t", bert_input,
      "\n\tbert_label:\t\t", bert_label)

print("-" * 100)

# Fija de nuevo la semilla para que coincidan los resultados
torch.manual_seed(100)

# Prepara los tensores incluyendo la lista de tokens sin procesar
bert_input, bert_label, raw_tokens_list = prepare_for_mlm(tokens, include_raw_tokens=True)

# Muestra los resultados con los tokens crudos
print("Con tokens crudos:\t",
      "\n\tEntrada original:\t", original_input,
      "\n\tbert_input:\t\t", bert_input,
      "\n\tbert_label:\t\t", bert_label,
      "\n\ttokens crudos:\t\t", raw_tokens_list)


Por lo tanto, a cada token en una oración se le asigna una etiqueta según la operación de enmascaramiento que se aplica sobre ese token. En este ejemplo, el primer "the" está **enmascarado**, por lo tanto, `bert_input` es `[MASK]` y su `bert_label` es `'The'`. 

Los tokens "sun", "sets", "behind" y el último "the" no se modifican, así que sus etiquetas correspondientes son `[PAD]`. “distant” está **enmascarado y reemplazado con un token aleatorio**, por lo tanto, `bert_input` es `[MASK]` y su `bert_label` es `'human-scaled'`. 

Finalmente, "mountains" y "." quedan **sin cambios**, por lo que sus etiquetas correspondientes son `[PAD]`.



#### **Preparación de datos para NSP**

La función `process_for_nsp` prepara los datos para la tarea de NSP creando pares de oraciones. Etiqueta estos pares para indicar si la segunda oración es la continuación de la primera en el texto original, facilitando que el modelo aprenda las relaciones entre oraciones.


In [ ]:
import random

def process_for_nsp(input_sentences, input_masked_labels):
    """
    Prepara los datos para la tarea NSP en el entrenamiento de BERT.

    Args:
    input_sentences (list): Lista de oraciones tokenizadas.
    input_masked_labels (list): Lista correspondiente de etiquetas enmascaradas para las oraciones.

    Returns:
    bert_input (list): Lista de pares de oraciones para la entrada de BERT.
    bert_label (list): Lista de etiquetas enmascaradas para los pares de oraciones.
    is_next (list): Lista de etiquetas binarias donde 1 indica 'siguiente oración' y 0 indica 'no siguiente oración'.
    """
    if len(input_sentences) < 2:
        raise ValueError("Debe haber al menos dos elementos.")

    # Verifica que ambas listas tengan la misma longitud
    if len(input_sentences) != len(input_masked_labels):
        raise ValueError("Ambas listas deben tener el mismo número de elementos.")

    bert_input = []
    bert_label = []
    is_next = []

    available_indices = list(range(len(input_sentences)))

    # Mientras haya al menos dos índices disponibles
    while len(available_indices) >= 2:
        if random.random() < 0.5:
            # Elige dos oraciones consecutivas para el escenario 'siguiente oración'
            index = random.choice(available_indices[:-1])  # Excluye el último índice
            # Añade los tokens '[CLS]' y '[SEP]' y construye el par de entrada
            bert_input.append(
                [
                    ['[CLS]'] + input_sentences[index] + ['[SEP]'],
                    input_sentences[index + 1] + ['[SEP]']
                ]
            )
            bert_label.append(
                [
                    ['[PAD]'] + input_masked_labels[index] + ['[PAD]'],
                    input_masked_labels[index + 1] + ['[PAD]']
                ]
            )
            is_next.append(1)  # Etiqueta 1 indica que son oraciones consecutivas

            # Elimina los índices usados
            available_indices.remove(index)
            if index + 1 in available_indices:
                available_indices.remove(index + 1)
        else:
            # Elige dos oraciones aleatorias no consecutivas para el escenario 'no siguiente oración'
            indices = random.sample(available_indices, 2)
            bert_input.append(
                [
                    ['[CLS]'] + input_sentences[indices[0]] + ['[SEP]'],
                    input_sentences[indices[1]] + ['[SEP]']
                ]
            )
            bert_label.append(
                [
                    ['[PAD]'] + input_masked_labels[indices[0]] + ['[PAD]'],
                    input_masked_labels[indices[1]] + ['[PAD]']
                ]
            )
            is_next.append(0)  # Etiqueta 0 indica que no son oraciones consecutivas

            # Elimina los índices usados
            available_indices.remove(indices[0])
            available_indices.remove(indices[1])

    return bert_input, bert_label, is_next


Veamos algunas oraciones de ejemplo y creemos pares NSP:



In [ ]:
# aplana el tensor
flatten = lambda l: [item for sublist in l for item in sublist]

# oraciones de entrada de ejemplo
input_sentences = [["i", "love", "apples"], ["she", "enjoys", "reading", "books"], ["he", "likes", "playing", "guitar"]]

# crea etiquetas enmascaradas para las oraciones
input_masked_labels = []
for sentence in input_sentences:
    _, current_masked_label = prepare_for_mlm(sentence, include_raw_tokens=False)
    input_masked_labels.append(flatten(current_masked_label))

# crea pares NSP y sus etiquetas
random.seed(100)
bert_input, bert_label, is_next = process_for_nsp(input_sentences, input_masked_labels)

# Imprime la salida
print("Entrada BERT:")
for pair in bert_input:
    print(pair)
print("Etiqueta BERT:")
for pair in bert_label:
    print(pair)
print("¿Es continuación?:", is_next)
print("-" * 200)

# Repite con otra semilla para la aleatoriedad
random.seed(1000)
bert_input, bert_label, is_next = process_for_nsp(input_sentences, input_masked_labels)

# Imprimir la salida nuevamente
print("Entrada BERT:")
for pair in bert_input:
    print(pair)
print("Etiqueta BERT:")
for pair in bert_label:
    print(pair)
print("¿Es continuación?:", is_next)


Estos dos ejemplos muestran cómo los pares de oraciones se crean de forma aleatoria a partir del banco de oraciones y se etiquetan para la tarea NSP. Primero se añaden los símbolos especiales \[CLS] y \[SEP] a las oraciones de entrada. La etiqueta de BERT se genera mediante la función `prepare_for_mlm`. 

En el primer ejemplo, la segunda oración sigue a la primera; por lo tanto, `Is Next` vale 1. En el segundo ejemplo, la segunda oración no sigue a la primera; así que `Is Next` vale 0.



#### **Finalizando las entradas de BERT**

`prepare_bert_final_inputs` consolida los datos preparados para MLM y NSP en un formato adecuado para el entrenamiento de BERT, incluyendo la conversión de tokens a índices, el relleno de secuencias para obtener una longitud uniforme y la generación de etiquetas de segmento para distinguir entre pares de oraciones. 

Esta función es el paso final en la preparación de los datos para BERT, asegurando que estén en el formato correcto para un entrenamiento efectivo del modelo.


In [25]:
from copy import deepcopy
import torch

def prepare_bert_final_inputs(bert_inputs, bert_labels, is_nexts, to_tenor=True):
    """
    Prepara las listas finales de entrada para el entrenamiento de BERT.
    """
    def zero_pad_list_pair(pair_, pad='[PAD]'):
        pair = deepcopy(pair_)
        max_len = max(len(pair[0]), len(pair[1]))
        # Añade [PAD] a cada frase del par hasta alcanzar la longitud máxima
        pair[0].extend([pad] * (max_len - len(pair[0])))
        pair[1].extend([pad] * (max_len - len(pair[1])))
        return pair[0], pair[1]

    # Aplana la lista de listas
    flatten = lambda l: [item for sublist in l for item in sublist]
    # Transforma tokens a índices del vocabulario
    tokens_to_index = lambda tokens: [vocab[token] for token in tokens]

    bert_inputs_final, bert_labels_final = [], []
    segment_labels_final, is_nexts_final = [], []

    for bert_input, bert_label, is_next in zip(bert_inputs, bert_labels, is_nexts):
        # Crea etiquetas de segmento para cada par de oraciones
        segment_label = [[1] * len(bert_input[0]), [2] * len(bert_input[1])]

        # Aplica padding con ceros a bert_input, bert_label y segment_label
        bert_input_padded = zero_pad_list_pair(bert_input)
        bert_label_padded = zero_pad_list_pair(bert_label)
        segment_label_padded = zero_pad_list_pair(segment_label, pad=0)

        # Convierte a tensores
        if to_tenor:
            # Aplana los pares con padding, convierte tokens a índices y crea tensores
            bert_inputs_final.append(
                torch.tensor(tokens_to_index(flatten(bert_input_padded)), dtype=torch.int64)
            )
            bert_labels_final.append(
                torch.tensor(tokens_to_index(flatten(bert_label_padded)), dtype=torch.int64)
            )
            segment_labels_final.append(
                torch.tensor(flatten(segment_label_padded), dtype=torch.int64)
            )
            is_nexts_final.append(is_next)
        else:
            # Solo aplana y deja los valores como listas
            bert_inputs_final.append(flatten(bert_input_padded))
            bert_labels_final.append(flatten(bert_label_padded))
            segment_labels_final.append(flatten(segment_label_padded))
            is_nexts_final.append(is_next)

    return bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final


Puedes comprobar los resultados usando `bert_input`, `bert_label` e `is_next` del ejemplo anterior:



In [ ]:
bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final = prepare_bert_final_inputs(
    bert_input,
    bert_label,
    is_next,
    to_tenor=True
)

torch.set_printoptions(linewidth=10000)  # esto asegura que toda la salida se imprima en una sola línea

print(
    "entrada:\t\t", bert_input,
    "\nentradas_finales:\t", bert_inputs_final,
    "\netiquetas_bert_finales:\t", bert_labels_final,
    "\netiquetas_segmento_finales:\t", segment_labels_final,
    "\nes_siguiente_final:\t", is_nexts_final
)


Las oraciones se rellenan con ceros y cada token se mapea a su índice en el vocabulario (`[CLS]` >> 1, `he` >> 33, ..., `[SEP]` >> 2, `[PAD]` >> 0).


Las etiquetas de máscara también se rellenan (padding) y se asignan a índices del vocabulario. En este caso, todos los tokens se **mantienen igual** excepto el token `he`, que está enmascarado:



In [ ]:
print("entrada:\t\t", bert_input,
      "\netiqueta_máscara:\t", bert_label,
      "\netiquetas_finales:\t", bert_labels_final)

Finalmente se crean las etiquetas de segmento: los tokens de la primera oración se etiquetan con 1, los de la segunda oración con 2, y los rellenos con ceros se etiquetan con 0.



In [ ]:
print(
    "\nentradas_finales:\t", bert_inputs_final,
    "\netiquetas_de_segmento:\t", segment_labels_final
)


### **Preparando el conjunto de datos**

* Se crea un archivo CSV para almacenar el conjunto de datos preparado para el entrenamiento y prueba de BERT. Cada fila contiene el texto original, las entradas para BERT, las etiquetas, las etiquetas de segmento y la etiqueta de la tarea NSP.
* Los datos del conjunto IMDB se tokenizan y se procesan primero para MLM y luego para NSP. Los resultados se formatean y se escriben en el archivo CSV, proporcionando un conjunto de datos completo para el entrenamiento del modelo BERT.

Este proceso es fundamental para garantizar que los datos estén en el formato adecuado para un entrenamiento eficaz de BERT con el conjunto IMDB, centrándose en comprender el contexto del texto y las relaciones entre oraciones.

> Este proceso de entrenamiento podría tardar alrededor de 2 a 3 horas.


In [ ]:
csv_file_path = 'train_bert_data_new.csv'
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['Original Text', 'BERT Input', 'BERT Label', 'Segment Label', 'Is Next'])

    # Envuelve train_iter con tqdm para mostrar una barra de progreso
    for n, (_, sample) in enumerate(tqdm(train_iter, desc="Procesando muestras")):
        # Tokeniza la entrada de la muestra
        tokens = tokenizer(sample)
        # Crea entradas y etiquetas para MLM
        bert_input, bert_label = prepare_for_mlm(tokens, include_raw_tokens=False)
        if len(bert_input) < 2:
            continue
        # Crea pares NSP, etiquetas de tokens y etiqueta is_next
        bert_inputs, bert_labels, is_nexts = process_for_nsp(bert_input, bert_label)
        # Añade relleno con ceros, mapear tokens a índices del vocabulario y crear etiquetas de segmento
        bert_inputs, bert_labels, segment_labels, is_nexts = prepare_bert_final_inputs(bert_inputs, bert_labels, is_nexts)
        # Convierte tensores a listas, listas a cadenas con formato JSON
        for bert_input, bert_label, segment_label, is_next in zip(bert_inputs, bert_labels, segment_labels, is_nexts):
            bert_input_str = json.dumps(bert_input.tolist())
            bert_label_str = json.dumps(bert_label.tolist())
            segment_label_str = ','.join(map(str, segment_label.tolist()))
            # Escribe los datos en el archivo CSV fila por fila
            csv_writer.writerow([sample, bert_input_str, bert_label_str, segment_label_str, is_next])


#### **Ejercicios**

Aprende a utilizar el BertTokenizer preentrenado de Hugging Face para la tokenización de texto, incluyendo el manejo de tokens especiales y la preparación del conjunto de datos IMDB para el entrenamiento de modelos de NLP, sin necesidad de construir manualmente un vocabulario.

#### **Ejercicio 1 - Inicialización del BERT Tokenizer**

1. **Importar `BertTokenizer`**:
   Comienza importando la clase `BertTokenizer` de la biblioteca `transformers`. Esta biblioteca ofrece acceso a una amplia gama de modelos de PLN y sus tokenizadores correspondientes.

2. **Cargar el tokenizador preentrenado**:
   Utiliza el método `from_pretrained` para cargar el tokenizador `bert-base-uncased`. Este tokenizador viene preconfigurado con un vocabulario adecuado para el modelo BERT entrenado con texto en inglés sin mayúsculas. Es ideal para comprender los fundamentos de la tokenización de BERT.



In [ ]:
from transformers import BertTokenizer

# Cargar un tokenizador BERT preentrenado
tokenizer = ...

#### **Ejercicio 2 - Tokenización del conjunto de datos**

1. **Define la función `yield_tokens`:**
   Implementa una función llamada `yield_tokens` que reciba un iterador sobre el conjunto de datos. Esta función será responsable de procesar y tokenizar los datos de texto.

2. **Tokeniza las muestras de texto:**
   Dentro de la función `yield_tokens`, recorre el conjunto de datos. Para cada muestra de texto, utiliza el `BertTokenizer` para convertir el texto en secuencias de IDs de tokens. Asegúrate de gestionar textos extensos estableciendo el parámetro `truncation=True` y especificando un `max_length` para que todas las salidas tokenizadas tengan un tamaño manejable.

3. **Devuelve los textos tokenizados:**
   Tras tokenizar cada muestra de texto, devuelve (yield) la lista de IDs de tokens. Estas listas se usarán en pasos posteriores para construir las estructuras de datos adecuadas para entrenar modelos de PLN.



In [ ]:
def yield_tokens(data_iter):
    ...

#### **Ejercicio 3 -Construcción del vocabulario con tokens especiales**

1. **Definir tokens especiales e índices**: Empieza definiendo índices para los tokens especiales como `[PAD]`, `[CLS]`, `[SEP]`, `[MASK]` y `[UNK]`. Crea una lista llamada `special_symbols` que incluya estos tokens, asegurándote de que estén en el orden correcto según sus índices.

2. **Cargar el dataset**: Asegúrate de tener cargada la partición de entrenamiento del dataset IMDB. Estos datos se usarán para construir el vocabulario.

3. **Construir el vocabulario**: Utiliza la función `build_vocab_from_iterator`, pasando como argumento la función generadora `yield_tokens`. Esta función recorre el dataset tokenizado y construye el vocabulario. Asegúrate de incluir los tokens especiales especificándolos en el parámetro `specials` de `build_vocab_from_iterator`.
   *(Como estás usando un tokenizador preentrenado, no necesitas construir el vocabulario desde cero; en su lugar, puedes usar directamente el vocabulario del tokenizador.)*

4. **Establecer el índice por defecto para tokens desconocidos**: Tras construir el vocabulario, usa el método `set_default_index` para asignar el índice correspondiente a los tokens desconocidos (`UNK_IDX`). Esto garantiza que cualquier token no encontrado en el vocabulario se maneje correctamente.
   *(De nuevo, si usas un tokenizador preentrenado, puedes prescindir de esta construcción y emplear directamente el vocabulario existente.)*


In [ ]:
from torchtext.data.functional import to_map_style_dataset
from torchtext.datasets import IMDB

...